### Plain Pandas

In [1]:
% % time
import ray

ray.init()

2022-01-28 15:00:28,144	INFO services.py:1338 -- View the Ray dashboard at http://127.0.0.1:8265


CPU times: total: 594 ms
Wall time: 8.39 s


{'node_ip_address': '127.0.0.1',
 'raylet_ip_address': '127.0.0.1',
 'redis_address': '127.0.0.1:6379',
 'object_store_address': 'tcp://127.0.0.1:59368',
 'raylet_socket_name': 'tcp://127.0.0.1:58994',
 'webui_url': '127.0.0.1:8265',
 'session_dir': 'C:\\Users\\Florian\\AppData\\Local\\Temp\\ray\\session_2022-01-28_15-00-23_239963_179476',
 'metrics_export_port': 61643,
 'node_id': 'efef5c9c53b351f1e349c229d2c51d6f525c097cb138f0f2633b285c'}

### Pandas

In [21]:
% % time
import pandas as pd

df = pd.read_csv("../sales-data.csv", parse_dates=["OrderDate"])

CPU times: total: 3.38 s
Wall time: 4.34 s


In [3]:
% % time
import pandas as pd

"""Top 10 customers"""
df.drop_duplicates(["CustomerId", "OrderId"]).groupby(by=["CustomerId"]).count().sort_values(by="OrderId",
                                                                                             ascending=False).head(10)[
    "OrderId"]

CPU times: total: 719 ms
Wall time: 972 ms


CustomerId
12835    11805
15       11141
9729     10239
14        9116
55        8914
12708     7915
69        7599
12823     7512
16        7471
3493      7455
Name: OrderId, dtype: int64

In [4]:
% % time

""" Top Sold Products per Month"""
df["OrderMonth"] = df["OrderDate"].dt.to_period('M')
df.groupby(by='OrderMonth').sum().sort_values(by="Quantity", ascending=False).head(10)["Quantity"]

CPU times: total: 906 ms
Wall time: 1.11 s


OrderMonth
2017-11    7.543293e+07
2017-03    7.459200e+07
2017-10    7.223356e+07
2017-06    7.191025e+07
2017-05    6.984081e+07
2018-06    6.936298e+07
2018-10    6.857377e+07
2018-02    6.821900e+07
2017-02    6.666570e+07
2018-03    6.664240e+07
Freq: M, Name: Quantity, dtype: float64

In [23]:
% % time

""" Produkte die pro Jahr am wenigsten gekauft wurden - nach Anzahl Orders von Produkt """
df["Year"] = df["OrderDate"].dt.to_period("Y")
years = df.sort_values(by="OrderDate")["Year"].unique()

for year in years:
    print("Year " + str(year))
    products_year = df.groupby(by=["Year"]).get_group(year)
    print(products_year.groupby(by=["ProductId"]).count().sort_values(by=["Quantity", "ProductId"]).head(10)["OrderId"])
    print()

Year 2015
ProductId
48222     1
116532    1
Name: Quantity, dtype: int64

Year 2016
ProductId
2     1
9     1
16    1
17    1
18    1
38    1
39    1
43    1
50    1
56    1
Name: Quantity, dtype: int64

Year 2017
ProductId
231     1
429     1
441     1
1014    1
1342    1
1413    1
1621    1
1663    1
1675    1
1709    1
Name: Quantity, dtype: int64

Year 2018
ProductId
204     1
248     1
429     1
469     1
593     1
853     1
935     1
939     1
978     1
1019    1
Name: Quantity, dtype: int64

CPU times: total: 3.11 s
Wall time: 3.74 s


In [24]:
% % time

""" Produkte die pro Jahr am wenigsten gekauft wurden - nach Summe der Quantity """
df["Year"] = df["OrderDate"].dt.to_period("Y")
years = df.sort_values(by="OrderDate")["Year"].unique()

for year in years:
    print("Year " + str(year))
    products_year = df.groupby(by=["Year"]).get_group(year)
    print(products_year.groupby(by=["ProductId"]).sum().sort_values(by=["Quantity", "ProductId"]).head(10)["Quantity"])

Year 2015
ProductId
48222      53.0
116532    191.0
Name: Quantity, dtype: float64
Year 2016
ProductId
16     1.0
17     1.0
39     1.0
72     1.0
75     1.0
91     1.0
98     1.0
116    1.0
158    1.0
174    1.0
Name: Quantity, dtype: float64
Year 2017
ProductId
88215     0.2
119806    0.2
121919    0.2
75428     0.4
4819      0.5
51682     0.5
88789     0.5
90193     0.5
113126    0.5
123489    0.5
Name: Quantity, dtype: float64
Year 2018
ProductId
144173    0.0
144174    0.0
13130     0.2
15821     0.2
119806    0.2
162387    0.2
99105     0.5
127957    0.5
141428    0.5
158591    0.5
Name: Quantity, dtype: float64
CPU times: total: 3.52 s
Wall time: 3.76 s


In [28]:
% % time
import datetime

""" Bestverkaufte Produkte in der Vorweihnachtszeit pro Jahr """
df["Year"] = df["OrderDate"].dt.to_period("Y")
years = df.sort_values(by="OrderDate")["Year"].unique()

for year in years:
    print("Year " + str(year))
    products_year = df.groupby(by=["Year"]).get_group(year)
    startdate = datetime.datetime(year.year, 11, 16)
    enddate = datetime.datetime(year.year, 12, 23)

    mask = (products_year["OrderDate"] >= startdate) & (products_year["OrderDate"] <= enddate)
    products_christmas_year = products_year.loc[mask]
    print(products_christmas_year.groupby(by=["ProductId"]).count().sort_values(by="Quantity", ascending=False).head(3))

2015
Empty DataFrame
Columns: [CustomerId, OrderId, ProductGroupId, Quantity, OrderDate, Year]
Index: []
2016
           CustomerId  OrderId  ProductGroupId  Quantity  OrderDate  Year
ProductId                                                                
22307             818      818             818       818        818   818
21346             302      302             302       302        302   302
22275             258      258             258       258        258   258
2017
           CustomerId  OrderId  ProductGroupId  Quantity  OrderDate  Year
ProductId                                                                
22307            1199     1199            1199      1199       1199  1199
21346             981      981             981       981        981   981
1289              372      372             372       372        372   372
2018
           CustomerId  OrderId  ProductGroupId  Quantity  OrderDate  Year
ProductId                                                         

### Modin Pandas with Ray

In [26]:
% % time
import modin.pandas as mpd

mdf = mpd.read_csv("../sales-data.csv", parse_dates=["OrderDate"])


CPU times: total: 266 ms
Wall time: 1.95 s


In [12]:
% % time

""" Top 10 Customers"""
mdf.drop_duplicates(["CustomerId", "OrderId"]).groupby(by=["CustomerId"]).count()["OrderId"]


CPU times: total: 9.52 s
Wall time: 37.3 s


CustomerId
1        267
2         22
3        591
4        572
5        516
        ... 
42653      1
42654      1
42655      1
42656      1
42657      1
Name: OrderId, Length: 42657, dtype: int64

In [13]:
% % time

""" Top Sold Products per Month"""
mdf["OrderMonth"] = mdf["OrderDate"].dt.to_period('M')
mdf.groupby(by='OrderMonth').sum().sort_values(by="Quantity", ascending=False).head(10)["Quantity"]


CPU times: total: 125 ms
Wall time: 3.19 s


To request implementation, send an email to feature_requests@modin.org.


OrderMonth
2017-11    7.543293e+07
2017-03    7.459200e+07
2017-10    7.223356e+07
2017-06    7.191025e+07
2017-05    6.984081e+07
2018-06    6.936298e+07
2018-10    6.857377e+07
2018-02    6.821900e+07
2017-02    6.666570e+07
2018-03    6.664240e+07
Name: Quantity, dtype: float64

In [18]:
% % time

""" Produkte die pro Jahr am wenigsten gekauft wurden  """
mdf["Year"] = mdf["OrderDate"].dt.to_period("Y")
years = mdf.sort_values(by="OrderDate")["Year"].unique()

for year in years:
    print("Year " + str(year))
    products_year = mdf.groupby(by=["Year"]).get_group(year)
    print(products_year.groupby(by=["ProductId"]).count().sort_values(by=["Quantity", "ProductId"]).head(10)["OrderId"])
    print()


Year 2015


ProductId
48222     1
116532    1
Name: OrderId, dtype: int64

Year 2016
ProductId
2     1
9     1
16    1
17    1
18    1
38    1
39    1
43    1
50    1
56    1
Name: OrderId, dtype: int64

Year 2017
ProductId
231     1
429     1
441     1
1014    1
1342    1
1413    1
1621    1
1663    1
1675    1
1709    1
Name: OrderId, dtype: int64

Year 2018
ProductId
204     1
248     1
429     1
469     1
593     1
853     1
935     1
939     1
978     1
1019    1
Name: OrderId, dtype: int64

CPU times: total: 5.23 s
Wall time: 10 s


In [29]:
% % time
import datetime

""" Bestverkaufte Produkte in der Vorweihnachtszeit pro Jahr """
mdf["Year"] = mdf["OrderDate"].dt.to_period("Y")
years = mdf.sort_values(by="OrderDate")["Year"].unique()

for year in years:
    print("Year " + str(year))
    products_year = mdf.groupby(by=["Year"]).get_group(year)
    startdate = datetime.datetime(year.year, 11, 16)
    enddate = datetime.datetime(year.year, 12, 23)

    mask = (products_year["OrderDate"] >= startdate) & (products_year["OrderDate"] <= enddate)
    products_christmas_year = products_year.loc[mask]
    print(products_christmas_year.groupby(by=["ProductId"]).count().sort_values(by="Quantity", ascending=False).head(3))

2015


Empty DataFrame
Columns: [CustomerId, OrderId, ProductGroupId, Quantity, OrderDate, Year]
Index: []
2016


           CustomerId  OrderId  ProductGroupId  Quantity  OrderDate  Year
ProductId                                                                
22307             818      818             818       818        818   818
21346             302      302             302       302        302   302
22275             258      258             258       258        258   258
2017


           CustomerId  OrderId  ProductGroupId  Quantity  OrderDate  Year
ProductId                                                                
22307            1199     1199            1199      1199       1199  1199
21346             981      981             981       981        981   981
1289              372      372             372       372        372   372
2018


           CustomerId  OrderId  ProductGroupId  Quantity  OrderDate  Year
ProductId                                                                
22307             643      643             643       643        643   643
40                437      437             437       437        437   437
682               408      408             408       408        408   408
CPU times: total: 6.52 s
Wall time: 10.1 s


In [32]:
% % time
""" Jährliche Wachstumsrate gemessen an den Order Line Items """
mdf["Year"] = mdf["OrderDate"].dt.to_period("Y")
years = mdf.sort_values(by="OrderDate")["Year"].unique()

year_lines = []

for year in years:
    products_year = mdf.groupby(by=["Year"]).get_group(year)

    amount_of_lines = products_year.count()["CustomerId"]
    year_lines.append((year.year, amount_of_lines))

print(year_lines)
year_growthrate_lines = []

prev = None
for i in range(len(year_lines)):
    if prev == None:
        year_growthrate_lines.append(
            ("Year: " + str(year_lines[i][0]), "100%", "Amount of lines: " + str(year_lines[i][1])))
        prev = year_lines[i]
    else:
        growthrate = (year_lines[i][1] / prev[1] - 1) * 100
        year_growthrate_lines.append(
            ("Year: " + str(year_lines[i][0]), str(int(growthrate)) + "%", "Amount of lines: " + str(year_lines[i][1])))
        prev = year_lines[i]

print(year_growthrate_lines)


[(2015, 2), (2016, 27376), (2017, 2628470), (2018, 2628375)]
[('Year: 2015', '100%', 'Amount of lines: 2'), ('Year: 2016', '1368700%', 'Amount of lines: 27376'), ('Year: 2017', '9501%', 'Amount of lines: 2628470'), ('Year: 2018', '0%', 'Amount of lines: 2628375')]
CPU times: total: 4.48 s
Wall time: 7.44 s


In [33]:
% % time

""" Jährliche Wachstumsrate gemessen an der Anzahl Orders """
mdf["Year"] = mdf["OrderDate"].dt.to_period("Y")
years = mdf.sort_values(by="OrderDate")["Year"].unique()
year_orders = []

for year in years:
    products_year = mdf.groupby(by=["Year"]).get_group(year)

    amount_of_orders = products_year["OrderId"].nunique()
    year_orders.append((year.year, amount_of_orders))

print(year_orders)
year_growthrate_orders = []

prev = None
for i in range(len(year_orders)):
    if prev is None:
        year_growthrate_orders.append(
            ("Year: " + str(year_orders[i][0]), "100%", "Amount of orders: " + str(year_orders[i][1])))
        prev = year_orders[i]
    else:
        growthrate = (year_orders[i][1] / prev[1] - 1) * 100
        year_growthrate_orders.append(("Year: " + str(year_orders[i][0]), str(int(growthrate)) + "%",
                                       "Amount of orders: " + str(year_orders[i][1])))
        prev = year_orders[i]

print(year_growthrate_orders)


[(2015, 1), (2016, 7387), (2017, 758969), (2018, 763670)]
[('Year: 2015', '100%', 'Amount of orders: 1'), ('Year: 2016', '738600%', 'Amount of orders: 7387'), ('Year: 2017', '10174%', 'Amount of orders: 758969'), ('Year: 2018', '0%', 'Amount of orders: 763670')]
CPU times: total: 4.78 s
Wall time: 8.79 s


In [36]:
%%time

""" Jährliche Wachstumsrate gemessen an der Quantity """
mdf["Year"] = mdf["OrderDate"].dt.to_period("Y")
years = mdf.sort_values(by="OrderDate")["Year"].unique()
year_quantities = []

for year in years:
    products_year = mdf.groupby(by=["Year"]).get_group(year)

    quantities = products_year["Quantity"].sum()
    year_quantities.append((year.year, quantities))
    #print(products_year.groupby(by=["ProductId"]).sum().sort_values(by=["Quantity", "ProductId"]).head(10))

print(year_quantities)
year_growthrate_quantities = []

prev = None
for i in range(len(year_quantities)):
    if prev is None:
        year_growthrate_quantities.append(
            ("Year: " + str(year_quantities[i][0]), "100%", "Sum of quantities: " + str(year_quantities[i][1])))
        prev = year_quantities[i]
    else:
        growthrate = (year_quantities[i][1] / prev[1] - 1) * 100
        year_growthrate_quantities.append(("Year: " + str(year_quantities[i][0]), str(int(growthrate)) + "%",
                                           "Sum of quantities: " + str(year_quantities[i][1])))
        prev = year_quantities[i]

print(year_growthrate_quantities)

[(2015, 244.0), (2016, 32456285.54), (2017, 800175941.4219999), (2018, 732054503.7)]
[('Year: 2015', '100%', 'Sum of quantities: 244.0'), ('Year: 2016', '13301656%', 'Sum of quantities: 32456285.54'), ('Year: 2017', '2365%', 'Sum of quantities: 800175941.4219999'), ('Year: 2018', '-8%', 'Sum of quantities: 732054503.7')]
CPU times: total: 4.73 s
Wall time: 9.65 s
